# Here we extract a uniformly sampled dataset of the environments to benchmark model accuracy off-sample for halfcheetah

In [ ]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import numpy as np 
import pandas as pd
from scipy.io import loadmat
import dotmap
from glob import glob
from dmbrl.modeling.models.NN import NN
from dmbrl.modeling.models.BNN import BNN
import json
import tensorflow as tf



import os
from time import localtime, strftime

from scipy.io import savemat
from dotmap import DotMap

from dmbrl.misc.DotmapUtils import get_required_argument
from dmbrl.misc.Agent import Agent
import copy
import numpy as np


import argparse
import pprint
import copy
from copy import deepcopy


from dotmap import DotMap

from dmbrl.misc.MBExp import MBExperiment
from dmbrl.controllers.MPC import MPC
from dmbrl.config import create_config
from dmbrl.misc import logger
from io import StringIO
import ast
import re

import time


def process_config_txt(config_txt,log_file):
    tf.reset_default_graph() 
    with open(config_txt,'r') as inf:
        k = inf.read()
        l = re.search(r'<.*<.*.>.*>',k).group(0)
        env_name = l.split('<')[-1].split('.')[0]
        print(env_name)
        readable = re.sub(r'<.*>',"\"\"",k)
    #     k = k.replace('0x','')
    #     k = k.replace('<',"\"")
    #     k = k.replace('>',"\"")

        dict_from_file = eval(readable)
        init_cfg = dict_from_file['ctrl_cfg']['prop_cfg']['model_init_cfg']
        activation = init_cfg['activation']
        network_shape = init_cfg['network_shape']
        weight_decays = init_cfg['weight_decays']
        lr = init_cfg['lr']
        overrides = [['ctrl_cfg.prop_cfg.model_init_cfg.activation', str(activation)], 
                     ['ctrl_cfg.prop_cfg.model_init_cfg.network_shape', str(network_shape)], 
                     ['ctrl_cfg.prop_cfg.model_init_cfg.lr',str(lr)], 
                     ['ctrl_cfg.prop_cfg.model_init_cfg.weight_decays',str(weight_decays)], 
                     ['exp_cfg.exp_cfg.ntrain_iters', '50'],
                     ['ctrl_cfg.prop_cfg.model_init_cfg.load_model','True'],
                     ['ctrl_cfg.prop_cfg.model_init_cfg.model_dir',log_file],
                    ['exp_cfg.exp_cfg.ntrain_iters',"1"]]
        logdir = 'log'
        ctrl_args = [['opt-type', 'CEM'], ['model-type', 'PE'], ['prop-type', 'E']]
        
        
        return dict_from_file,overrides,logdir,ctrl_args,env_name


for environment_name in ['ant','halfcheetah','hopper']:
    # log_dir = '/home/motion/Joao/classes/ece598sg-mps/final_project_ece598/POPLIN/log/test_hopper/'
    # log_dir = '/home/motion/Joao/classes/ece598sg-mps/final_project_ece598/POPLIN/log/test_halfcheetah/'
    # log_dir = '/home/motion/Joao/classes/ece598sg-mps/final_project_ece598/POPLIN/log/test_ant/'
    log_dir = '/home/motion/Joao/classes/ece598sg-mps/final_project_ece598/POPLIN/log/test_{}/'.format(environment_name)
    config_file = glob(log_dir+'*.txt')[0]
    # with open(config_raw[0],'r') as inf:
    #     k = inf.read()
    #     l = re.search(r'<.*<.*.>.*>',k).group(0)
    #     env = l.split('<')[-1].split('.')[0]
    #     readable = re.sub(r'<.*>',"\"\"",k)
    # #     k = k.replace('0x','')
    # #     k = k.replace('<',"\"")
    # #     k = k.replace('>',"\"")

    #     dict_from_file = eval(readable)
    processed_dict,overrides,logdir,ctrl_args,env_name = process_config_txt(config_file,log_dir)
    ctrl_args = DotMap(**{key: val for (key, val) in ctrl_args})
    cfg = create_config(env_name,"MPC", ctrl_args, overrides, logdir)
    cfg.exp_cfg.exp_cfg.policy = MPC(cfg.ctrl_cfg)


    cfg.exp_cfg.misc = copy.copy(cfg)
    exp = MBExperiment(cfg.exp_cfg)
    if(env_name != 'halfcheetah'):
        print('this is ',env_name)
        env = exp.env._env.env
    else:
        print('we are working with {}'.format(env_name))
        env = exp.env
    # we add a function that allows for arbitrary positioning of the model:
    # def place_model(self,init_qpos = [],init_qvel = []):
    #     if(len(init_qpos)!= 0):
    #         if(len(init_qvel !=0)):
    #             qpos = init_qpos
    #             qvel = init_qvel
    #         else:
    #             qpos = self.init_qpos + self.np_random.uniform(size=self.model.nq, low=-.1, high=.1) 
    #             qvel = self.init_qvel + self.np_random.randn(self.model.nv) * .1
    #     else:
    #         qpos = self.init_qpos + self.np_random.uniform(size=self.model.nq, low=-.1, high=.1)
    #         qvel = self.init_qvel + self.np_random.randn(self.model.nv) * .1
    #     self.set_state(qpos, qvel)

    #     return self._get_obs()

    # env.place_model = place_model
    model = env.model

    # we then get the model qbounds - and set the qbounds on the joints:

    # we then obtain the joint limits and sample 50.000 examples within the limits

    # We then set the ranges according to what we saw during all the experiments:

    if(env_name == 'halfcheetah'):
        high = [9.96128474,0.98984282,56.42344112,1.10883563,0.89405376,0.91599938,0.88814824,
                                 1.07776409,0.73160817,10.48143774,4.97410966,13.39867047,
                                 25.49660014,31.22417231,
                                 24.50860937,31.08020618, 33.91224822,
                                 29.45049987] 
        low = [ 0,-0.60089228,-3.38180708,-0.70808846,-0.94726328,-0.69846955,-1.1664694,-1.25208849,-0.71914243,
                                                        -4.62570942,-6.07998072,-10.07353218,-27.25994739,-29.93182322,
                                                        -29.46358498, -26.98998881,-32.6241283,-23.90487897]
    elif(env_name == 'gym_ant'):
        high = [ 2.95740289,0.9999914,0.99982883,0.9998585,0.99980208,0.68677208,1.35133411,
                0.6962879,0.09895299,0.68500648,0.09878973,0.69019431,1.36160244,
                4.74811162,4.5842259,6.27479401,14.2021727,12.43167361,10.69651593,
                17.07403083,18.01675964,17.01246318,14.53401165,17.38114028,14.47999146,
                17.0391048,17.98908219]
        low = [0.22637723 ,-0.99981976,-0.99996822,-0.99974723,-0.99999862,
      -0.68875781,-0.0986322,-0.68763238,-1.35011608,-0.68757495,
      -1.35376862,  -0.68970977,  -0.09800097,  -4.14476645,  -4.48196699,
      -5.98612387, -13.08248117, -11.78631638, -12.34927498, -17.55141179,
     -14.24190613, -17.32666853, -15.94113497, -17.47971411, -17.23695828,
     -16.91510594, -14.06285142]
    elif(env_name == 'gym_hopper'):
        high = [ 1.96040359,14.67674145,  0.24221648,  0.20641022,  0.97567123,  7.98996199,
      7.60698399, 34.36266375, 31.65726622, 27.55551616, 25.28091824]
        low = [ 1.03056874e-02, -1.56137981e+01, -2.84094996e+00, -2.79574698e+00,
     -9.66243311e-01, -7.24971707e+00, -1.22857845e+01, -4.70124229e+01,
     -2.94250451e+01, -2.35840978e+01, -2.38191301e+01]


    inputs = []
    outputs = []
    for i in range(100000):
    #     pos = np.random.uniform(low = ranges[:,0],high = ranges[:,1])
    #     vel = np.random.uniform(low = ranges[:,0],high = ranges[:,1])
        env.reset()
        vec = np.random.uniform(high = high , 
                          low = low)
    #     if(env_name == 'halfcheetah'):
        q_pos = env.init_qpos
        rem = len(q_pos)-len(vec) + len(env.init_qvel)
        q_pos[rem:env.init_qpos.shape[0]] = vec[rem:env.init_qpos.shape[0]]
        q_vel = vec[-len(env.init_qvel):]
    #     else:
        env.set_state(qpos = q_pos,qvel = q_vel)
        init_pos = env._get_obs()
        if(env_name == 'gym_ant'):
            init_pos = init_pos[:27]

    #     print(env._get_obs(),vecs)
        act = np.random.uniform(high = env.action_space.high,low = env.action_space.low)
        inputs.append(init_pos.tolist()+act.tolist())
        output = env.step(act)[0].tolist()
        if(env_name == 'gym_ant'):
            output = output[:27]
        outputs.append(output)

    inputs = np.array(inputs)
    outputs = np.array(outputs)
    np.save('./gt_datasets/{}/outputs'.format(env_name),outputs)
    np.save('./gt_datasets/{}/inputs'.format(env_name),inputs)

# Loading a network

In [ ]:
network_params = DotMap()


In [ ]:
tf.reset_default_graph() 
network_params.load_model = True
network_params.model_dir = '/home/motion/Joao/classes/ece598sg-mps/final_project_ece598/POPLIN/log/test_ant/'
network_params.name = 'model'
nw = BNN(network_params)
nw.finalize(tf.train.AdamOptimizer, {"learning_rate": 0.001})

# Loading the dataset

In [ ]:
outputs = np.load('./gt_datasets/gym_ant/outputs.npy')
inputs = np.load('./gt_datasets//inputs.npy')

In [ ]:
%%time
pred_outs = nw.predict(inputs)

diff = outputs - pred_outs[0]
diff = np.power(diff,2)
(diff.mean())

In [ ]:
pred_outs[0].shape

In [ ]:
outputs.shape

# Automating the analysis of the data:

In [ ]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import pandas as pd
from scipy.io import loadmat
import dotmap
from glob import glob
from dmbrl.modeling.models.NN import NN
from dmbrl.modeling.models.BNN import BNN
import json
import tensorflow as tf
import os
from time import localtime, strftime
from scipy.io import savemat
from dmbrl.misc.DotmapUtils import get_required_argument
from dmbrl.misc.Agent import Agent
import numpy as np
import pprint
import copy
from copy import deepcopy
from dotmap import DotMap
from dmbrl.misc.MBExp import MBExperiment
from dmbrl.controllers.MPC import MPC
from dmbrl.config import create_config
from dmbrl.misc import logger
from io import StringIO
import ast
import re

import time

In [ ]:
# loading all the log files:

exp_dirs = glob('./results/*/*/*/')

oos_errors = []
error_stds = []
exp_names  = []
env_names  = []
max_returns = []
max_returns_std = []
is_errors = []
for exp_dir in exp_dirs:
    a = 1
    seed_dirs = glob(exp_dir+'*/')
    environment = exp_dir.split('/')[-2]
    exp_name = exp_dir.split('/')[-3]
    gt_outputs = np.load('./gt_datasets/{}/outputs.npy'.format(environment))
    inputs = np.load('./gt_datasets/{}/inputs.npy'.format(environment))
    errors = []
    returns = []
    for seed_dir in seed_dirs:
        tf.reset_default_graph() 
        network_params = DotMap()
        network_params.load_model = True
        network_params.model_dir = seed_dir
        network_params.name = 'model'
        nw = BNN(network_params)
        nw.finalize(tf.train.AdamOptimizer, {"learning_rate": 0.001})
        training_logs = loadmat(seed_dir+'logs.mat')
        returns = training_logs['returns'].max()
        pred_outs = nw.predict(inputs)
        diff = gt_outputs - pred_outs[0]
        diff = np.power(diff,2)
        errors = diff.mean()
        # finally we calculate the in-sample errors:
        obs = training_logs['observations']
        in_sample_obs = obs[:,:-1,:].reshape(-1,obs.shape[-1])
        in_sample_gt = obs[:,1:,:].reshape(-1,obs.shape[-1])
        acts = training_logs['actions']
        acts = acts.reshape(-1,acts.shape[-1])
        in_sample_inputs = np.concatenate((in_sample_obs,acts),axis = 1)
        in_preds = nw.predict(in_sample_inputs)
        in_sample_diffs = in_sample_gt - in_preds[0]
        in_sample_diffs = np.power(in_sample_diffs,2)
        in_sample_error = np.mean(in_sample_diffs)
        
        oos_errors.append(np.mean(errors))
#         error_stds.append(np.std(errors))
        max_returns.append(np.mean(returns))
#         max_returns_std.append(np.std(returns))
        exp_names.append(exp_name)
        env_names.append(environment)
        is_errors.append(in_sample_error)

In [ ]:
obs = training_logs['observations']
in_sample_obs = obs[:,:-1,:].reshape(-1,obs.shape[-1])
in_sample_gt = obs[:,1:,:].reshape(-1,obs.shape[-1])
acts = training_logs['actions']
acts = acts.reshape(-1,acts.shape[-1])
in_sample_inputs = np.concatenate((in_sample_obs,acts),axis = 1)

In [ ]:

performance_dataset = pd.DataFrame({'oos_error':oos_errors,'is_error':is_errors,'max_returns':max_returns,'exp_names':exp_names,'env_names':env_names})

In [ ]:
performance_dataset

# Plotting OOS error vs rewards

In [ ]:
env_names = performance_dataset.env_names.unique()
for env_name in env_names:
    tmp1 = performance_dataset[performance_dataset.env_names == env_name]
    exp_names = tmp1.exp_names.unique()
    for exp_name in exp_names:
        tmp = tmp1[tmp1.exp_names == exp_name]
        plt.scatter(tmp.oos_error.values.tolist(),tmp.max_returns.values.tolist(),label = exp_name[6:])
        plt.xscale('log')
    plt.legend()
    plt.title('Out of Sample Error x Max Returns in-sample - {}'.format(env_name))
    plt.xlim(left = 1)
    plt.ylim(bottom = np.min([0,tmp1.max_returns.min()]))
    plt.xlabel('log(Out of Sample MSE)')
    plt.ylabel('Max Train Return')
    plt.savefig('./figures/Out of Sample Error x Max Returns in-sample - {}.pdf'.format(env_name),bbox_inches = 'tight')
    plt.show()


# Plotting IS_ERROR VS OOS_ERROR

In [ ]:
env_names = performance_dataset.env_names.unique()
for env_name in env_names:
    tmp1 = performance_dataset[performance_dataset.env_names == env_name]
    exp_names = tmp1.exp_names.unique()
    for exp_name in exp_names:
        tmp = tmp1[tmp1.exp_names == exp_name]
        plt.scatter(tmp.is_error.values.tolist(),tmp.oos_error.values.tolist(),label = exp_name[6:])
        plt.xscale('log')
        plt.yscale('log')
    plt.legend()
    plt.title('In-sample error x Out of Sample Error - {}'.format(env_name))
    plt.xlim(left = 1)
    plt.ylim(bottom = 1)
    plt.xlabel('log(In Sample MSE)')
    plt.ylabel('log(Out of Sample MSE)')
    plt.savefig('./figures/In-Sample Error x Out of Sample Error - {}.pdf'.format(env_name),bbox_inches = 'tight')
    plt.show()

# Plotting In-sample error vs max_returns

In [ ]:
env_names = performance_dataset.env_names.unique()
for env_name in env_names:
    tmp1 = performance_dataset[performance_dataset.env_names == env_name]
    exp_names = tmp1.exp_names.unique()
    for exp_name in exp_names:
        tmp = tmp1[tmp1.exp_names == exp_name]
        plt.scatter(tmp.is_error.values.tolist(),tmp.max_returns.values.tolist(),label = exp_name[6:])
        plt.xscale('log')
    plt.legend()
    plt.title('In-Sample Error x Max Returns in-sample - {}'.format(env_name))
    plt.xlim(left = 1)
    plt.ylim(bottom = np.min([0,tmp1.max_returns.min()]))
    plt.xlabel('log(In-Sample MSE)')
    plt.ylabel('Max Train Return')
    plt.savefig('./figures/In-Sample Error x Max Returns in-sample - {}.pdf'.format(env_name),bbox_inches = 'tight')
    plt.show()


# Generating Videos for the presentation

In [4]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import numpy as np 
import pandas as pd
from scipy.io import loadmat
import dotmap
from glob import glob
from dmbrl.modeling.models.NN import NN
from dmbrl.modeling.models.BNN import BNN
import json
import tensorflow as tf



import os
from time import localtime, strftime

from scipy.io import savemat
from dotmap import DotMap

from dmbrl.misc.DotmapUtils import get_required_argument
from dmbrl.misc.Agent import Agent
import copy
import numpy as np


import argparse
import pprint
import copy
from copy import deepcopy


from dotmap import DotMap

from dmbrl.misc.MBExp import MBExperiment
from dmbrl.controllers.MPC import MPC
from dmbrl.config import create_config
from dmbrl.misc import logger
from io import StringIO
import ast
import re

import time


def process_config_txt(config_txt,log_file):
    tf.reset_default_graph() 
    with open(config_txt,'r') as inf:
        k = inf.read()
        l = re.search(r'<.*<.*.>.*>',k).group(0)
        env_name = l.split('<')[-1].split('.')[0]
        print(env_name)
        readable = re.sub(r'<.*>',"\"\"",k)
    #     k = k.replace('0x','')
    #     k = k.replace('<',"\"")
    #     k = k.replace('>',"\"")

        dict_from_file = eval(readable)
        init_cfg = dict_from_file['ctrl_cfg']['prop_cfg']['model_init_cfg']
        try:
            activation = init_cfg['activation']
        except:
            activation = 'swish'
        try:
            network_shape = init_cfg['network_shape']
        except:
            network_shape = '[200,200,200,200]'
        try:
            weight_decays = init_cfg['weight_decays']
        except:
            weight_decays = '[0.000025,0.00005,0.000075,0.000075,0.0001]'
        try:
            lr = init_cfg['lr']
        except:
            lr = '0.001'
        overrides = [['ctrl_cfg.prop_cfg.model_init_cfg.activation', str(activation)], 
                     ['ctrl_cfg.prop_cfg.model_init_cfg.network_shape', str(network_shape)], 
                     ['ctrl_cfg.prop_cfg.model_init_cfg.lr',str(lr)], 
                     ['ctrl_cfg.prop_cfg.model_init_cfg.weight_decays',str(weight_decays)], 
                     ['exp_cfg.exp_cfg.ntrain_iters', '50'],
                     ['ctrl_cfg.prop_cfg.model_init_cfg.load_model','True'],
                     ['ctrl_cfg.prop_cfg.model_init_cfg.model_dir',log_file],
                    ['exp_cfg.exp_cfg.ntrain_iters',"1"]]
        logdir = 'log'
        ctrl_args = [['opt-type', 'CEM'], ['model-type', 'PE'], ['prop-type', 'E']]
        
        
        return dict_from_file,overrides,logdir,ctrl_args,env_name


# for environment_name in ['ant','halfcheetah','hopper']:
exp_dirs = glob('./results/*/*/*/')

for exp_dir in exp_dirs[:]:
    a = 1
    seed_dirs = glob(exp_dir+'*/')
    environment = exp_dir.split('/')[-2]
    exp_name = exp_dir.split('/')[-3]
    gt_outputs = np.load('./gt_datasets/{}/outputs.npy'.format(environment))
    inputs = np.load('./gt_datasets/{}/inputs.npy'.format(environment))
    errors = []
    returns = []
    for seed,seed_dir in enumerate(seed_dirs):
        movie_dir = './videos/{}/{}'.format(environment,exp_name)
        if not os.path.exists(movie_dir):
            os.mkdir(movie_dir)
        movie_file = movie_dir+'/{}.mp4'.format(seed)
        if(not os.path.isfile(movie_file)):
            log_dir = seed_dir
            config_file = glob(log_dir+'*.txt')[0]
            # with open(config_raw[0],'r') as inf:
            #     k = inf.read()
            #     l = re.search(r'<.*<.*.>.*>',k).group(0)
            #     env = l.split('<')[-1].split('.')[0]
            #     readable = re.sub(r'<.*>',"\"\"",k)
            # #     k = k.replace('0x','')
            # #     k = k.replace('<',"\"")
            # #     k = k.replace('>',"\"")

            #     dict_from_file = eval(readable)
            processed_dict,overrides,logdir,ctrl_args,env_name = process_config_txt(config_file,log_dir)
            ctrl_args = DotMap(**{key: val for (key, val) in ctrl_args})
            cfg = create_config(env_name,"MPC", ctrl_args, overrides, logdir)
            cfg.exp_cfg.exp_cfg.policy = MPC(cfg.ctrl_cfg)


            cfg.exp_cfg.misc = copy.copy(cfg)
            exp = MBExperiment(cfg.exp_cfg)
            if(env_name != 'halfcheetah'):
                print('this is ',env_name)
                env = exp.env._env
                exp.agent.env.metadata = env.metadata
                exp.agent.env.render = env.render
    #             print(exp.agent.env._env_info['max_length'])
    #             exp.agent.env = env
    #             exp.agent.env render= env.
            else:
                print('we are working with {}'.format(env_name))
                env = exp.env


            # env.place_model = place_model
    #         model = env.model
    #         if(environment == 'gym_hopper'):
    #             exp.agent.sample(horizon = 100000,policy = exp.policy,record_fname = movie_file)
    #         else:
            exp.agent.sample(horizon = 1000,policy = exp.policy,record_fname = movie_file)
            time.sleep(3)
        else:
            print('movie file = {} already exists'.format(movie_file))
#         break
#     if(environment == 'gym_hopper'):
#         break



movie file = ./videos/halfcheetah/layer_2x200_swish/0.mp4 already exists
movie file = ./videos/halfcheetah/layer_2x200_swish/1.mp4 already exists
movie file = ./videos/halfcheetah/layer_2x200_swish/2.mp4 already exists
movie file = ./videos/gym_ant/layer_2x200_swish/0.mp4 already exists
movie file = ./videos/gym_ant/layer_2x200_swish/1.mp4 already exists
movie file = ./videos/gym_ant/layer_2x200_swish/2.mp4 already exists
movie file = ./videos/gym_hopper/layer_2x200_swish/0.mp4 already exists
movie file = ./videos/gym_hopper/layer_2x200_swish/1.mp4 already exists
movie file = ./videos/gym_hopper/layer_2x200_swish/2.mp4 already exists
movie file = ./videos/halfcheetah/layer_4x200_sine/0.mp4 already exists
movie file = ./videos/halfcheetah/layer_4x200_sine/1.mp4 already exists
movie file = ./videos/halfcheetah/layer_4x200_sine/2.mp4 already exists
movie file = ./videos/gym_ant/layer_4x200_sine/0.mp4 already exists
movie file = ./videos/gym_ant/layer_4x200_sine/1.mp4 already exists
movie 

In [ ]:
exp.policy